In [0]:
from pyspark.sql.functions import lit

# Load 2008 data
benef_2008 = (
    spark.read
    .parquet("/mnt/bronze/beneficiary/Beneficiary_Summary_File_2008.parquet")
    .withColumn("YEAR", lit(2008))
)

# Load 2009 data
benef_2009 = (
    spark.read
    .parquet("/mnt/bronze/beneficiary/Beneficiary_Summary_File_2009.parquet")
    .withColumn("YEAR", lit(2009))
)

# Load 2010 data
benef_2010 = (
    spark.read
    .parquet("/mnt/bronze/beneficiary/Beneficiary_Summary_File_2010.parquet")
    .withColumn("YEAR", lit(2010))
)

# Union all years
beneficiary_all_years = benef_2008.unionByName(benef_2009).unionByName(benef_2010)

# Display the result
display(beneficiary_all_years)



In [0]:
beneficiary_all_years.write.mode("overwrite").parquet("/mnt/silver/beneficiary/beneficiary_all_years")

In [0]:
beneficiary = spark.read.parquet("/mnt/silver/beneficiary/beneficiary_all_years")

Beneficiary: Converts birth and death dates to proper date format, calculates age as of 2010, derives AGE_GROUP, removes duplicates, fills nulls, and standardizes column names to lowercase

In [0]:
from pyspark.sql.functions import to_date, col, year, lit, when
def standardize_columns(df):

    return df.toDF(*[c.lower() for c in df.columns])

# Clean Beneficiary

beneficiary_clean = (
    beneficiary.dropDuplicates(["DESYNPUF_ID"])
    .withColumn("BENE_BIRTH_DT", to_date(col("BENE_BIRTH_DT"), "yyyyMMdd"))
    .withColumn("BENE_DEATH_DT", to_date(col("BENE_DEATH_DT"), "yyyyMMdd"))
    .withColumn("AGE", lit(2010) - year(col("BENE_BIRTH_DT")))  
    .withColumn(
        "AGE_GROUP",
        when(col("AGE") < 70, "<70")
        .when((col("AGE") >= 70) & (col("AGE") < 75), "70-74")
        .when((col("AGE") >= 75) & (col("AGE") < 80), "75-79")
        .otherwise("80+")
    )
    .na.drop(subset=["BENE_BIRTH_DT"])  # critical column
    .fillna(0)  # fill other nulls
)

beneficiary_clean = standardize_columns(beneficiary_clean)

In [0]:
beneficiary_clean.write.mode("overwrite").parquet("/mnt/silver/beneficiary/beneficiary_clean")

In [0]:
# Beneficiary Silver Table
beneficiary_filtered = beneficiary_clean.select(
    "desynpuf_id", "bene_birth_dt", "bene_death_dt",
    "bene_sex_ident_cd", "bene_race_cd",
    "age", "age_group",
    "sp_alzhdmta", "sp_chf", "sp_chrnkidn", "sp_cncr",
    "sp_copd", "sp_depressn", "sp_diabetes", "sp_ischmcht",
    "sp_osteoprs", "sp_ra_oa", "sp_strketia"
)

In [0]:
beneficiary_filtered.write.mode("overwrite").parquet("/mnt/silver/beneficiary/beneficiary_filtered")

In [0]:
from pyspark.sql.functions import when

beneficiary_features = (
    beneficiary_clean
    .withColumn(
        "IS_HIGH_COST_PATIENT",
        when(
            (col("MEDREIMB_IP") + col("MEDREIMB_OP") + col("MEDREIMB_CAR")) > 10000,
            1
        ).otherwise(0)
    )
)

In [0]:
beneficiary_features.write.mode("overwrite").parquet("/mnt/silver/beneficiary/beneficiary_features")